In [1]:
import asyncio
import awswrangler as wr
from datetime import datetime, timedelta
from IPython.display import display
from metaapi_cloud_sdk import MetaApi
import os
import pandas as pd
import pytz
import sys
from time import sleep

In [2]:
repo_name = "mercury-pipeline-metaapi-sam"
sys.path.append(os.getcwd().split(repo_name)[0] + repo_name + "/lambda") 

In [3]:
from src.config import *
from src.io import *
from src.utils import *
from src.access import get_demo_account

In [4]:
from src.ticks.clean.get import get_clean_ticks
from src.candles.from_ticks.get import get_candles_from_ticks
from src.candles.transform.save import ffill

In [5]:
async def yield_raw_ticks_between_datetimes(
    account: object,
    symbol: str,
    datetimeutc_from: datetime,
    datetimeutc_to: datetime,
    pause: int = 5
):
    offset = 0
    while True:
        l_ticks_i = await account.get_historical_ticks(
            symbol,
            start_time=datetimeutc_from,
            limit=1000,
            offset=offset,
        )
        if len(l_ticks_i) == 0:
            break
        yield offset, l_ticks_i
        await asyncio.sleep(pause)
        datetimeutc_last = l_ticks_i[-1]["time"]
        offset += len(l_ticks_i)
        if datetimeutc_last >= datetimeutc_to:
            break

In [6]:
symbol = "GBPUSD"
frequency = "15S"

In [7]:
datetimestr_from = (
    pytz
    .timezone("Europe/London")
    .localize(datetime.now())
    .astimezone(pytz.timezone("UTC"))
    .strftime("%Y%m%d000000000000")
)

datetimestr_to = (
    pytz
    .timezone("Europe/London")
    .localize(datetime.now() + timedelta(days=1))
    .astimezone(pytz.timezone("UTC"))
    .strftime("%Y%m%d%H%M%S%f")
)
datetimestr_from, datetimestr_to

('20220708000000000000', '20220709151119939699')

In [8]:
# Get demo account
account = await get_demo_account()

# Convert datetimes from string to datetime
datetimeutc_from = pytz.timezone("UTC").localize(datetime.strptime(datetimestr_from, "%Y%m%d%H%M%S%f"))
datetimeutc_to = pytz.timezone("UTC").localize(datetime.strptime(datetimestr_to, "%Y%m%d%H%M%S%f"))

In [9]:
generator_async = yield_raw_ticks_between_datetimes(
    account=account,
    symbol=symbol,
    datetimeutc_from=datetimeutc_from,
    datetimeutc_to=datetimeutc_to,
)

In [10]:
i = 0
df_ticks_clean_prev = pd.DataFrame()
async for offset, ticks_raw in generator_async:
    
    print(offset)
    
    if ticks_raw == []:
        sleep(10)
        continue

    df_ticks_clean = get_clean_ticks(ticks_raw)
    df_ticks_clean = pd.concat([
        df_ticks_clean_prev,
        df_ticks_clean
    ])
    
    df_candles_from_ticks = get_candles_from_ticks(
        df_ticks_clean,
        frequency
    )

    df_candles_ffill = ffill(df_candles_from_ticks)
    
    s_candle_last = df_candles_ffill.iloc[-1]
    df_candles_out = df_candles_ffill.iloc[:-1].copy()
    
    mask_last = df_ticks_clean["timestamp_utc"] >= s_candle_last["timestamp_utc.open"]
    df_ticks_clean_prev = df_ticks_clean[mask_last].copy()
    
    display(df_ticks_clean)

    i += 1
    if i >= 3:
        break
    sleep(1)

TypeError: 'async_generator' object is not iterable

In [ ]:
assert False

In [11]:
offset, ticks_raw = await generator_async.__anext__()

In [12]:
ticks_raw[0], ticks_raw[-1]

({'symbol': 'GBPUSD',
  'time': datetime.datetime(2022, 7, 8, 0, 0, 0, 83000, tzinfo=datetime.timezone.utc),
  'last': 1.20271,
  'volume': 150000,
  'brokerTime': '2022-07-08 03:00:00.083'},
 {'symbol': 'GBPUSD',
  'time': datetime.datetime(2022, 7, 8, 0, 1, 58, 539000, tzinfo=datetime.timezone.utc),
  'last': 1.20321,
  'volume': 150000,
  'brokerTime': '2022-07-08 03:01:58.539'})

In [13]:
df_ticks_clean = get_clean_ticks(ticks_raw)
df_ticks_clean

,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
0,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.083,2022-07-08 01:00:00.083,2022-07-08 03:00:00.083,NaN,NaN,1.20271,150000.0
1,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.083,2022-07-08 01:00:00.083,2022-07-08 03:00:00.083,1.20271,1.20321,NaN,NaN
2,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.131,2022-07-08 01:00:00.131,2022-07-08 03:00:00.131,NaN,NaN,1.20291,150000.0
3,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.132,2022-07-08 01:00:00.132,2022-07-08 03:00:00.132,1.20291,1.20303,NaN,NaN
4,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:00:00.234,2022-07-08 01:00:00.234,2022-07-08 03:00:00.234,NaN,NaN,1.20290,150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.135,2022-07-08 01:01:58.135,2022-07-08 03:01:58.135,NaN,NaN,1.20321,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.236,2022-07-08 01:01:58.236,2022-07-08 03:01:58.236,NaN,NaN,1.20321,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.337,2022-07-08 01:01:58.337,2022-07-08 03:01:58.337,NaN,NaN,1.20321,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.438,2022-07-08 01:01:58.438,2022-07-08 03:01:58.438,NaN,NaN,1.20321,150000.0


In [14]:
df_candles_from_ticks = get_candles_from_ticks(
    df_ticks_clean,
    frequency
)
df_candles_from_ticks

,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close
0,2022-07-08 00:00:00,1.20321,1.20325,1.20302,1.20327,32,1.20271,1.20313,1.20271,1.20315,lo,lo,2022-07-08 00:00:15,2022-07-08 01:00:00,2022-07-08 01:00:15
1,2022-07-08 00:00:15,1.20320,1.20334,1.20319,1.20335,23,1.20308,1.20322,1.20307,1.20323,lo,lo,2022-07-08 00:00:30,2022-07-08 01:00:15,2022-07-08 01:00:30
2,2022-07-08 00:00:30,1.20335,1.20328,1.20328,1.20335,9,1.20323,1.20316,1.20316,1.20323,hi,hi,2022-07-08 00:00:45,2022-07-08 01:00:30,2022-07-08 01:00:45
3,2022-07-08 00:00:45,1.20326,1.20325,1.20321,1.20327,10,1.20314,1.20313,1.20309,1.20315,lo,lo,2022-07-08 00:01:00,2022-07-08 01:00:45,2022-07-08 01:01:00
4,2022-07-08 00:01:00,1.20322,1.20331,1.20322,1.20331,12,1.20313,1.20319,1.20310,1.20319,lo,lo,2022-07-08 00:01:15,2022-07-08 01:01:00,2022-07-08 01:01:15
5,2022-07-08 00:01:15,1.20330,1.20332,1.20325,1.20336,22,1.20318,1.20320,1.20313,1.20324,lo,lo,2022-07-08 00:01:30,2022-07-08 01:01:15,2022-07-08 01:01:30
6,2022-07-08 00:01:30,1.20330,1.20332,1.20327,1.20335,17,1.20318,1.20320,1.20314,1.20323,lo,lo,2022-07-08 00:01:45,2022-07-08 01:01:30,2022-07-08 01:01:45
7,2022-07-08 00:01:45,1.20333,1.20333,1.20333,1.20333,1,1.20321,1.20321,1.20321,1.20321,eq,eq,2022-07-08 00:02:00,2022-07-08 01:01:45,2022-07-08 01:02:00


In [15]:
df_candles_ffill = ffill(df_candles_from_ticks)
df_candles_ffill

,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close
0,2022-07-08 00:00:00,1.20321,1.20325,1.20302,1.20327,32,1.20271,1.20313,1.20271,1.20315,lo,lo,2022-07-08 00:00:15,2022-07-08 01:00:00,2022-07-08 01:00:15
1,2022-07-08 00:00:15,1.20320,1.20334,1.20319,1.20335,23,1.20308,1.20322,1.20307,1.20323,lo,lo,2022-07-08 00:00:30,2022-07-08 01:00:15,2022-07-08 01:00:30
2,2022-07-08 00:00:30,1.20335,1.20328,1.20328,1.20335,9,1.20323,1.20316,1.20316,1.20323,hi,hi,2022-07-08 00:00:45,2022-07-08 01:00:30,2022-07-08 01:00:45
3,2022-07-08 00:00:45,1.20326,1.20325,1.20321,1.20327,10,1.20314,1.20313,1.20309,1.20315,lo,lo,2022-07-08 00:01:00,2022-07-08 01:00:45,2022-07-08 01:01:00
4,2022-07-08 00:01:00,1.20322,1.20331,1.20322,1.20331,12,1.20313,1.20319,1.20310,1.20319,lo,lo,2022-07-08 00:01:15,2022-07-08 01:01:00,2022-07-08 01:01:15
5,2022-07-08 00:01:15,1.20330,1.20332,1.20325,1.20336,22,1.20318,1.20320,1.20313,1.20324,lo,lo,2022-07-08 00:01:30,2022-07-08 01:01:15,2022-07-08 01:01:30
6,2022-07-08 00:01:30,1.20330,1.20332,1.20327,1.20335,17,1.20318,1.20320,1.20314,1.20323,lo,lo,2022-07-08 00:01:45,2022-07-08 01:01:30,2022-07-08 01:01:45
7,2022-07-08 00:01:45,1.20333,1.20333,1.20333,1.20333,1,1.20321,1.20321,1.20321,1.20321,eq,eq,2022-07-08 00:02:00,2022-07-08 01:01:45,2022-07-08 01:02:00


In [16]:
s_candle_last = df_candles_ffill.iloc[-1]
s_candle_last

timestamp_utc.open     2022-07-08 00:01:45
ask.open                           1.20333
ask.close                          1.20333
ask.low                            1.20333
ask.high                           1.20333
n_ticks                                  1
bid.open                           1.20321
bid.close                          1.20321
bid.low                            1.20321
bid.high                           1.20321
ask.idx_hi_lo                           eq
bid.idx_hi_lo                           eq
timestamp_utc.close    2022-07-08 00:02:00
timestamp_uk.open      2022-07-08 01:01:45
timestamp_uk.close     2022-07-08 01:02:00
Name: 7, dtype: object

In [17]:
df_candles_out = df_candles_ffill.iloc[:-1].copy()
df_candles_out

,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close
0,2022-07-08 00:00:00,1.20321,1.20325,1.20302,1.20327,32,1.20271,1.20313,1.20271,1.20315,lo,lo,2022-07-08 00:00:15,2022-07-08 01:00:00,2022-07-08 01:00:15
1,2022-07-08 00:00:15,1.20320,1.20334,1.20319,1.20335,23,1.20308,1.20322,1.20307,1.20323,lo,lo,2022-07-08 00:00:30,2022-07-08 01:00:15,2022-07-08 01:00:30
2,2022-07-08 00:00:30,1.20335,1.20328,1.20328,1.20335,9,1.20323,1.20316,1.20316,1.20323,hi,hi,2022-07-08 00:00:45,2022-07-08 01:00:30,2022-07-08 01:00:45
3,2022-07-08 00:00:45,1.20326,1.20325,1.20321,1.20327,10,1.20314,1.20313,1.20309,1.20315,lo,lo,2022-07-08 00:01:00,2022-07-08 01:00:45,2022-07-08 01:01:00
4,2022-07-08 00:01:00,1.20322,1.20331,1.20322,1.20331,12,1.20313,1.20319,1.20310,1.20319,lo,lo,2022-07-08 00:01:15,2022-07-08 01:01:00,2022-07-08 01:01:15
5,2022-07-08 00:01:15,1.20330,1.20332,1.20325,1.20336,22,1.20318,1.20320,1.20313,1.20324,lo,lo,2022-07-08 00:01:30,2022-07-08 01:01:15,2022-07-08 01:01:30
6,2022-07-08 00:01:30,1.20330,1.20332,1.20327,1.20335,17,1.20318,1.20320,1.20314,1.20323,lo,lo,2022-07-08 00:01:45,2022-07-08 01:01:30,2022-07-08 01:01:45


In [18]:
df_ticks_clean_last = df_ticks_clean[df_ticks_clean["timestamp_utc"] >= s_candle_last["timestamp_utc.open"]].copy()
df_ticks_clean_last

,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
926,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.050,2022-07-08 01:01:45.050,2022-07-08 03:01:45.050,NaN,NaN,1.20320,150000.0
927,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.151,2022-07-08 01:01:45.151,2022-07-08 03:01:45.151,NaN,NaN,1.20320,150000.0
928,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.253,2022-07-08 01:01:45.253,2022-07-08 03:01:45.253,NaN,NaN,1.20320,150000.0
929,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.527,2022-07-08 01:01:45.527,2022-07-08 03:01:45.527,NaN,NaN,1.20320,150000.0
930,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.636,2022-07-08 01:01:45.636,2022-07-08 03:01:45.636,NaN,NaN,1.20320,150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.135,2022-07-08 01:01:58.135,2022-07-08 03:01:58.135,NaN,NaN,1.20321,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.236,2022-07-08 01:01:58.236,2022-07-08 03:01:58.236,NaN,NaN,1.20321,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.337,2022-07-08 01:01:58.337,2022-07-08 03:01:58.337,NaN,NaN,1.20321,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.438,2022-07-08 01:01:58.438,2022-07-08 03:01:58.438,NaN,NaN,1.20321,150000.0


In [19]:
offset, ticks_raw = await generator_async.__anext__()

In [20]:
df_ticks_clean = get_clean_ticks(ticks_raw)
df_ticks_clean

,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
0,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.700,2022-07-08 01:01:58.700,2022-07-08 03:01:58.700,NaN,NaN,1.20321,150000.0
1,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.802,2022-07-08 01:01:58.802,2022-07-08 03:01:58.802,NaN,NaN,1.20322,150000.0
2,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.802,2022-07-08 01:01:58.802,2022-07-08 03:01:58.802,1.20322,1.20334,NaN,NaN
3,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.905,2022-07-08 01:01:58.905,2022-07-08 03:01:58.905,NaN,NaN,1.20325,150000.0
4,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:58.906,2022-07-08 01:01:58.906,2022-07-08 03:01:58.906,1.20325,1.20337,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.041,2022-07-08 01:04:07.041,2022-07-08 03:04:07.041,NaN,NaN,1.20332,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.142,2022-07-08 01:04:07.142,2022-07-08 03:04:07.142,NaN,NaN,1.20332,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.420,2022-07-08 01:04:07.420,2022-07-08 03:04:07.420,NaN,NaN,1.20332,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.519,2022-07-08 01:04:07.519,2022-07-08 03:04:07.519,NaN,NaN,1.20332,150000.0


In [21]:
df_ticks_clean = pd.concat([
    df_ticks_clean_last,
    df_ticks_clean
])
df_ticks_clean

,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
926,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.050,2022-07-08 01:01:45.050,2022-07-08 03:01:45.050,NaN,NaN,1.20320,150000.0
927,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.151,2022-07-08 01:01:45.151,2022-07-08 03:01:45.151,NaN,NaN,1.20320,150000.0
928,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.253,2022-07-08 01:01:45.253,2022-07-08 03:01:45.253,NaN,NaN,1.20320,150000.0
929,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.527,2022-07-08 01:01:45.527,2022-07-08 03:01:45.527,NaN,NaN,1.20320,150000.0
930,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:01:45.636,2022-07-08 01:01:45.636,2022-07-08 03:01:45.636,NaN,NaN,1.20320,150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.041,2022-07-08 01:04:07.041,2022-07-08 03:04:07.041,NaN,NaN,1.20332,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.142,2022-07-08 01:04:07.142,2022-07-08 03:04:07.142,NaN,NaN,1.20332,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.420,2022-07-08 01:04:07.420,2022-07-08 03:04:07.420,NaN,NaN,1.20332,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.519,2022-07-08 01:04:07.519,2022-07-08 03:04:07.519,NaN,NaN,1.20332,150000.0


In [22]:
df_candles_from_ticks = get_candles_from_ticks(
    df_ticks_clean,
    frequency
)
df_candles_from_ticks

,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close
0,2022-07-08 00:01:45,1.20333,1.20337,1.20333,1.20337,3,1.20321,1.20325,1.20321,1.20325,lo,lo,2022-07-08 00:02:00,2022-07-08 01:01:45,2022-07-08 01:02:00
1,2022-07-08 00:02:00,1.20335,1.20328,1.20327,1.20335,9,1.20325,1.20316,1.20315,1.20325,hi,hi,2022-07-08 00:02:15,2022-07-08 01:02:00,2022-07-08 01:02:15
2,2022-07-08 00:02:15,1.20327,1.20324,1.20324,1.20328,4,1.20315,1.20312,1.20312,1.20316,hi,hi,2022-07-08 00:02:30,2022-07-08 01:02:15,2022-07-08 01:02:30
3,2022-07-08 00:02:30,1.20328,1.20340,1.20328,1.20340,11,1.20316,1.20328,1.20316,1.20328,lo,lo,2022-07-08 00:02:45,2022-07-08 01:02:30,2022-07-08 01:02:45
4,2022-07-08 00:02:45,1.20339,1.20334,1.20333,1.20339,6,1.20327,1.20322,1.20321,1.20327,hi,hi,2022-07-08 00:03:00,2022-07-08 01:02:45,2022-07-08 01:03:00
5,2022-07-08 00:03:00,1.20336,1.20337,1.20335,1.20346,17,1.20322,1.20325,1.20322,1.20334,lo,lo,2022-07-08 00:03:15,2022-07-08 01:03:00,2022-07-08 01:03:15
6,2022-07-08 00:03:15,1.20338,1.20355,1.20337,1.20359,23,1.20326,1.20343,1.20325,1.20347,lo,lo,2022-07-08 00:03:30,2022-07-08 01:03:15,2022-07-08 01:03:30
7,2022-07-08 00:03:30,1.20354,1.20347,1.20347,1.20359,14,1.20342,1.20335,1.20335,1.20347,hi,hi,2022-07-08 00:03:45,2022-07-08 01:03:30,2022-07-08 01:03:45
8,2022-07-08 00:03:45,1.20348,1.20353,1.20342,1.20354,11,1.20336,1.20341,1.20330,1.20342,lo,lo,2022-07-08 00:04:00,2022-07-08 01:03:45,2022-07-08 01:04:00
9,2022-07-08 00:04:00,1.20349,1.20344,1.20344,1.20350,5,1.20341,1.20332,1.20332,1.20341,hi,hi,2022-07-08 00:04:15,2022-07-08 01:04:00,2022-07-08 01:04:15


In [23]:
df_candles_ffill = ffill(df_candles_from_ticks)
df_candles_ffill

,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close
0,2022-07-08 00:01:45,1.20333,1.20337,1.20333,1.20337,3,1.20321,1.20325,1.20321,1.20325,lo,lo,2022-07-08 00:02:00,2022-07-08 01:01:45,2022-07-08 01:02:00
1,2022-07-08 00:02:00,1.20335,1.20328,1.20327,1.20335,9,1.20325,1.20316,1.20315,1.20325,hi,hi,2022-07-08 00:02:15,2022-07-08 01:02:00,2022-07-08 01:02:15
2,2022-07-08 00:02:15,1.20327,1.20324,1.20324,1.20328,4,1.20315,1.20312,1.20312,1.20316,hi,hi,2022-07-08 00:02:30,2022-07-08 01:02:15,2022-07-08 01:02:30
3,2022-07-08 00:02:30,1.20328,1.20340,1.20328,1.20340,11,1.20316,1.20328,1.20316,1.20328,lo,lo,2022-07-08 00:02:45,2022-07-08 01:02:30,2022-07-08 01:02:45
4,2022-07-08 00:02:45,1.20339,1.20334,1.20333,1.20339,6,1.20327,1.20322,1.20321,1.20327,hi,hi,2022-07-08 00:03:00,2022-07-08 01:02:45,2022-07-08 01:03:00
5,2022-07-08 00:03:00,1.20336,1.20337,1.20335,1.20346,17,1.20322,1.20325,1.20322,1.20334,lo,lo,2022-07-08 00:03:15,2022-07-08 01:03:00,2022-07-08 01:03:15
6,2022-07-08 00:03:15,1.20338,1.20355,1.20337,1.20359,23,1.20326,1.20343,1.20325,1.20347,lo,lo,2022-07-08 00:03:30,2022-07-08 01:03:15,2022-07-08 01:03:30
7,2022-07-08 00:03:30,1.20354,1.20347,1.20347,1.20359,14,1.20342,1.20335,1.20335,1.20347,hi,hi,2022-07-08 00:03:45,2022-07-08 01:03:30,2022-07-08 01:03:45
8,2022-07-08 00:03:45,1.20348,1.20353,1.20342,1.20354,11,1.20336,1.20341,1.20330,1.20342,lo,lo,2022-07-08 00:04:00,2022-07-08 01:03:45,2022-07-08 01:04:00
9,2022-07-08 00:04:00,1.20349,1.20344,1.20344,1.20350,5,1.20341,1.20332,1.20332,1.20341,hi,hi,2022-07-08 00:04:15,2022-07-08 01:04:00,2022-07-08 01:04:15


In [24]:
s_candle_last = df_candles_ffill.iloc[-1]
s_candle_last

timestamp_utc.open     2022-07-08 00:04:00
ask.open                           1.20349
ask.close                          1.20344
ask.low                            1.20344
ask.high                            1.2035
n_ticks                                  5
bid.open                           1.20341
bid.close                          1.20332
bid.low                            1.20332
bid.high                           1.20341
ask.idx_hi_lo                           hi
bid.idx_hi_lo                           hi
timestamp_utc.close    2022-07-08 00:04:15
timestamp_uk.open      2022-07-08 01:04:00
timestamp_uk.close     2022-07-08 01:04:15
Name: 9, dtype: object

In [25]:
df_candles_out = df_candles_ffill.iloc[:-1].copy()
df_candles_out

,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close
0,2022-07-08 00:01:45,1.20333,1.20337,1.20333,1.20337,3,1.20321,1.20325,1.20321,1.20325,lo,lo,2022-07-08 00:02:00,2022-07-08 01:01:45,2022-07-08 01:02:00
1,2022-07-08 00:02:00,1.20335,1.20328,1.20327,1.20335,9,1.20325,1.20316,1.20315,1.20325,hi,hi,2022-07-08 00:02:15,2022-07-08 01:02:00,2022-07-08 01:02:15
2,2022-07-08 00:02:15,1.20327,1.20324,1.20324,1.20328,4,1.20315,1.20312,1.20312,1.20316,hi,hi,2022-07-08 00:02:30,2022-07-08 01:02:15,2022-07-08 01:02:30
3,2022-07-08 00:02:30,1.20328,1.20340,1.20328,1.20340,11,1.20316,1.20328,1.20316,1.20328,lo,lo,2022-07-08 00:02:45,2022-07-08 01:02:30,2022-07-08 01:02:45
4,2022-07-08 00:02:45,1.20339,1.20334,1.20333,1.20339,6,1.20327,1.20322,1.20321,1.20327,hi,hi,2022-07-08 00:03:00,2022-07-08 01:02:45,2022-07-08 01:03:00
5,2022-07-08 00:03:00,1.20336,1.20337,1.20335,1.20346,17,1.20322,1.20325,1.20322,1.20334,lo,lo,2022-07-08 00:03:15,2022-07-08 01:03:00,2022-07-08 01:03:15
6,2022-07-08 00:03:15,1.20338,1.20355,1.20337,1.20359,23,1.20326,1.20343,1.20325,1.20347,lo,lo,2022-07-08 00:03:30,2022-07-08 01:03:15,2022-07-08 01:03:30
7,2022-07-08 00:03:30,1.20354,1.20347,1.20347,1.20359,14,1.20342,1.20335,1.20335,1.20347,hi,hi,2022-07-08 00:03:45,2022-07-08 01:03:30,2022-07-08 01:03:45
8,2022-07-08 00:03:45,1.20348,1.20353,1.20342,1.20354,11,1.20336,1.20341,1.20330,1.20342,lo,lo,2022-07-08 00:04:00,2022-07-08 01:03:45,2022-07-08 01:04:00


In [26]:
df_ticks_clean_last = df_ticks_clean[df_ticks_clean["timestamp_utc"] >= s_candle_last["timestamp_utc.open"]].copy()
df_ticks_clean_last

,symbol,frequency,year,month,date,hour,timestamp_utc,timestamp_uk,timestamp_broker,bid,ask,last,volume
938,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:00.047,2022-07-08 01:04:00.047,2022-07-08 03:04:00.047,NaN,NaN,1.20341,150000.0
939,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:00.047,2022-07-08 01:04:00.047,2022-07-08 03:04:00.047,1.20341,NaN,NaN,NaN
940,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:00.148,2022-07-08 01:04:00.148,2022-07-08 03:04:00.148,NaN,NaN,1.20341,150000.0
941,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:00.247,2022-07-08 01:04:00.247,2022-07-08 03:04:00.247,NaN,NaN,1.20341,150000.0
942,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:00.350,2022-07-08 01:04:00.350,2022-07-08 03:04:00.350,NaN,NaN,1.20341,150000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.041,2022-07-08 01:04:07.041,2022-07-08 03:04:07.041,NaN,NaN,1.20332,150000.0
996,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.142,2022-07-08 01:04:07.142,2022-07-08 03:04:07.142,NaN,NaN,1.20332,150000.0
997,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.420,2022-07-08 01:04:07.420,2022-07-08 03:04:07.420,NaN,NaN,1.20332,150000.0
998,GBPUSD,tick,2022,07,20220708,00,2022-07-08 00:04:07.519,2022-07-08 01:04:07.519,2022-07-08 03:04:07.519,NaN,NaN,1.20332,150000.0
